In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

from datetime import datetime, timedelta

util_dir = os.path.join(os.getcwd(), r'../..', 'util')
stckinfo_dir = os.path.join(os.getcwd(), r'../..', 'util', 'stocks_info')

sys.path.append(util_dir)
sys.path.append(stckinfo_dir)


In [3]:
import kis_auth as ka
import kis_domstk as kb

from kis_kospi_code_mst import kospi_master_download, get_kospi_master_dataframe

ka.auth()

Done


In [18]:
data_dir = os.path.join(os.path.expanduser('~'), 'data','ofij')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [13]:
import time

import pandas as pd
from tqdm import tqdm

In [5]:
dfkospicc = pd.read_csv('kospi_code.csv')

In [9]:
stockcodes = dfkospicc[dfkospicc.그룹코드 == 'ST'].단축코드.tolist()

In [14]:
stock_info_results = {}

for stock_code in tqdm(stockcodes, desc="Fetching stock info"):
    stock_info = kb.get_search_stock_info(pdno=stock_code)
    stock_info_results[stock_code] = stock_info
    time.sleep(0.5)


stock_info_df = pd.DataFrame.from_dict(stock_info_results, orient='index')

# save to feather file
stock_info_df.reset_index(inplace=True)
stock_info_df.rename(columns={'index': '단축코드'}, inplace=True)
stock_info_df.to_feather(os.path.join(data_dir, 'stock_info.feather'))

Fetching stock info: 100%|██████████| 932/932 [24:02<00:00,  1.55s/it]


In [ ]:
lst_stock_prices = []

period_codes = ['D', 'W', 'M']
for period_code in period_codes:
    print(f"Fetching stock prices for period {period_code}")
    for stock_code in tqdm(stockcodes, desc=f"Fetching stock prices for period {period_code}"):
        stock_prices = kb.get_inquire_daily_price(itm_no = stock_code, period_code=period_code)
        # add preiod code to stock_prices dataframe
        stock_prices['PERIOD_CODE'] = period_code
        # add stock_code to stock_prices dataframe if not exist
        if 'STOCK_CODE' not in stock_prices.columns:
            stock_prices['STOCK_CODE'] = stock_code
        lst_stock_prices.append(stock_prices)

# concatenate all stock prices dataframes
stock_prices = pd.concat(lst_stock_prices, ignore_index=True)
# save to feather file
stock_prices.reset_index(inplace=True, drop=True)
stock_prices.to_feather(os.path.join(data_dir, 'stock_prices.feather'))

Fetching stock prices for period D


Fetching stock prices for period D: 100%|██████████| 932/932 [15:54<00:00,  1.02s/it]


Fetching stock prices for period W


Fetching stock prices for period W: 100%|██████████| 932/932 [16:13<00:00,  1.04s/it]


Fetching stock prices for period M


Fetching stock prices for period M: 100%|██████████| 932/932 [16:23<00:00,  1.05s/it]

